In [1]:
from pathlib import Path
import MeCab
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from transformers import AutoTokenizer, BertJapaneseTokenizer
from src.utils import load_dataset, display_score
from src.train import MultiLabelBERTTrainer

In [2]:
# CONFIG
TRAIN_DATA_DIR = Path('../data/train')
SEED = 2023
FOLDS = 5
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
EPOCHS = 100
MAX_LENGTH = 512
EARLY_STOPPING_ROUNDS = 5
CRITERION = nn.CrossEntropyLoss()

# Read Data
df = load_dataset(TRAIN_DATA_DIR)
df['TNM'] = df['T'].astype(str) + df['N'].astype(str) + df['M'].astype(str)
fold = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

In [3]:
# 東北大BERT-v2
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

cv = fold.split(df['text'], df['TNM'])
trainer = MultiLabelBERTTrainer(model_name=MODEL_NAME, tokenizer=TOKENIZER, criterion=CRITERION, device=DEVICE, seed=SEED)
cv_predsTNM = trainer.training_cv(df, cv, batch_size=BATCH_SIZE, epochs=EPOCHS, learning_rate=LEARNING_RATE, max_length=MAX_LENGTH, early_stopping_rounds=EARLY_STOPPING_ROUNDS)
display_score(df['TNM'].values, cv_predsTNM)

/home/is/takuya-f/NTCIR17/.venv/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly ident

BEST Epochs: [12, 15, 12, 18, 11]
macro F1: 0.7325 Accuracy: 0.7963
T | macro F1: 0.6947 Accuracy: 0.7381
N | macro F1: 0.6211 Accuracy: 0.7540
M | macro F1: 0.8817 Accuracy: 0.8968
TNM | macro F1: 0.2508 Accuracy: 0.5476


In [4]:
# UTH-BERT
MODEL_NAME = '../data/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K'
TOKENIZER = BertJapaneseTokenizer.from_pretrained(MODEL_NAME, mecab_kwargs={'mecab_option': '-d ../data/mecab-unidic-neologd -u ../data/MANBYO_201907_Dic-utf8.dic'})

cv = fold.split(df['text'], df['TNM'])
trainer = MultiLabelBERTTrainer(model_name=MODEL_NAME, tokenizer=TOKENIZER, criterion=CRITERION, device=DEVICE, seed=SEED)
cv_predsTNM = trainer.training_cv(df, cv, batch_size=BATCH_SIZE, epochs=EPOCHS, learning_rate=LEARNING_RATE, max_length=MAX_LENGTH, early_stopping_rounds=EARLY_STOPPING_ROUNDS)
display_score(df['TNM'].values, cv_predsTNM)

/home/is/takuya-f/NTCIR17/.venv/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
Some weights of the model checkpoint at ../data/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to

BEST Epochs: [25, 15, 19, 21, 14]
macro F1: 0.7132 Accuracy: 0.8016
T | macro F1: 0.6713 Accuracy: 0.7143
N | macro F1: 0.5474 Accuracy: 0.7619
M | macro F1: 0.9210 Accuracy: 0.9286
TNM | macro F1: 0.2430 Accuracy: 0.5238


In [5]:
# JMedRoBERTa (sentencepiece)
MODEL_NAME = 'alabnii/jmedroberta-base-sentencepiece-vocab50000'
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

cv = fold.split(df['text'], df['TNM'])
trainer = MultiLabelBERTTrainer(model_name=MODEL_NAME, tokenizer=TOKENIZER, criterion=CRITERION, device=DEVICE, seed=SEED)
cv_predsTNM = trainer.training_cv(df, cv, batch_size=BATCH_SIZE, epochs=EPOCHS, learning_rate=LEARNING_RATE, max_length=MAX_LENGTH, early_stopping_rounds=EARLY_STOPPING_ROUNDS)
display_score(df['TNM'].values, cv_predsTNM)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/is/takuya-f/NTCIR17/.venv/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
Some weights of the model checkpoint at alabnii/jmedroberta-base-sentencepiece-vocab50000 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing 

BEST Epochs: [12, 14, 11, 12, 11]
macro F1: 0.7100 Accuracy: 0.7725
T | macro F1: 0.5886 Accuracy: 0.6508
N | macro F1: 0.6401 Accuracy: 0.7540
M | macro F1: 0.9012 Accuracy: 0.9127
TNM | macro F1: 0.2069 Accuracy: 0.4524


In [6]:
# JMedRoBERTa (manbyo-wordpiece)
MODEL_NAME = 'alabnii/jmedroberta-base-manbyo-wordpiece-vocab50000'
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME, **{
    'mecab_kwargs': {
        'mecab_option': '-u ../data/MANBYO_201907_Dic-utf8.dic'
    }
})

cv = fold.split(df['text'], df['TNM'])
trainer = MultiLabelBERTTrainer(model_name=MODEL_NAME, tokenizer=TOKENIZER, criterion=CRITERION, device=DEVICE, seed=SEED)
cv_predsTNM = trainer.training_cv(df, cv, batch_size=BATCH_SIZE, epochs=EPOCHS, learning_rate=LEARNING_RATE, max_length=MAX_LENGTH, early_stopping_rounds=EARLY_STOPPING_ROUNDS)
display_score(df['TNM'].values, cv_predsTNM)

/home/is/takuya-f/NTCIR17/.venv/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
Some weights of the model checkpoint at alabnii/jmedroberta-base-manbyo-wordpiece-vocab50000 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCla

BEST Epochs: [14, 16, 15, 15, 20]
macro F1: 0.7429 Accuracy: 0.8069
T | macro F1: 0.6621 Accuracy: 0.7143
N | macro F1: 0.6464 Accuracy: 0.7778
M | macro F1: 0.9201 Accuracy: 0.9286
TNM | macro F1: 0.2762 Accuracy: 0.5317


In [7]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

cv = fold.split(df['text'], df['TNM'])
trainer = MultiLabelBERTTrainer(model_name=MODEL_NAME, tokenizer=TOKENIZER, criterion=CRITERION, device=DEVICE, seed=SEED)
cv_predsTNM = trainer.training_cv(df, cv, batch_size=BATCH_SIZE, epochs=EPOCHS, learning_rate=LEARNING_RATE, max_length=MAX_LENGTH, early_stopping_rounds=EARLY_STOPPING_ROUNDS)
display_score(df['TNM'].values, cv_predsTNM)

/home/is/takuya-f/NTCIR17/.venv/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v3 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly ident

BEST Epochs: [17, 14, 12, 11, 12]
macro F1: 0.7403 Accuracy: 0.7884
T | macro F1: 0.6844 Accuracy: 0.6905
N | macro F1: 0.6414 Accuracy: 0.7698
M | macro F1: 0.8952 Accuracy: 0.9048
TNM | macro F1: 0.2592 Accuracy: 0.5159


In [8]:
# 東北大BERT-v3-char
MODEL_NAME = 'cl-tohoku/bert-base-japanese-char-v3'
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

cv = fold.split(df['text'], df['TNM'])
trainer = MultiLabelBERTTrainer(model_name=MODEL_NAME, tokenizer=TOKENIZER, criterion=CRITERION, device=DEVICE, seed=SEED)
cv_predsTNM = trainer.training_cv(df, cv, batch_size=BATCH_SIZE, epochs=EPOCHS, learning_rate=LEARNING_RATE, max_length=MAX_LENGTH, early_stopping_rounds=EARLY_STOPPING_ROUNDS)
display_score(df['TNM'].values, cv_predsTNM)

/home/is/takuya-f/NTCIR17/.venv/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-char-v3 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly 

BEST Epochs: [21, 25, 11, 13, 15]
macro F1: 0.7417 Accuracy: 0.8201
T | macro F1: 0.7415 Accuracy: 0.8016
N | macro F1: 0.5543 Accuracy: 0.7222
M | macro F1: 0.9294 Accuracy: 0.9365
TNM | macro F1: 0.2498 Accuracy: 0.5952
